In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

In [ ]:
import os
for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/logos"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 9 directories and 0 images in '/content/drive/MyDrive/logos'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Volkswagen'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Suzuki'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Nissan'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Honday'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Hyundai'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Tata'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Renault'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Ford'.
There are 0 directories and 45 images in '/content/drive/MyDrive/logos/Toyota'.


#PRE-AUGMENTATION

In [ ]:
data_dir='/content/drive/MyDrive/logos'

In [ ]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 383 files belonging to 9 classes.
Using 307 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 383 files belonging to 9 classes.
Using 76 files for validation.


In [ ]:
class_names = val_ds.class_names
print(class_names)

['Ford', 'Honday', 'Hyundai', 'Nissan', 'Renault', 'Suzuki', 'Tata', 'Toyota', 'Volkswagen']


#RESNET50 MODEL

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=9,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(9, activation='softmax'))

94765736/94765736 [==============================] - 3s 0us/step


In [ ]:
resnet_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
history = resnet_model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/10
10/10 [==============================] - 64s 6s/step - loss: 0.0514 - accuracy: 1.0000 - val_loss: 1.7819 - val_accuracy: 0.4868
Epoch 2/10
10/10 [==============================] - 63s 6s/step - loss: 0.0436 - accuracy: 1.0000 - val_loss: 1.8032 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 63s 6s/step - loss: 0.0361 - accuracy: 1.0000 - val_loss: 1.7741 - val_accuracy: 0.5000
Epoch 4/10
10/10 [==============================] - 61s 6s/step - loss: 0.0304 - accuracy: 1.0000 - val_loss: 1.8036 - val_accuracy: 0.5263
Epoch 5/10
10/10 [==============================] - 63s 6s/step - loss: 0.0249 - accuracy: 1.0000 - val_loss: 1.8691 - val_accuracy: 0.5132
Epoch 6/10
10/10 [==============================] - 62s 6s/step - loss: 0.0206 - accuracy: 1.0000 - val_loss: 1.7942 - val_accuracy: 0.5132
Epoch 7/10
10/10 [==============================] - 53s 5s/step - loss: 0.0186 - accuracy: 1.0000 - val_loss: 1.8672 - val_accuracy: 0.5263
Epoch 8/10
10/10 [==

#VGG16

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
import tensorflow as tf

In [ ]:
vgg_model = Sequential()
pretrained_model = VGG16(include_top=False,
                         input_shape=(180, 180, 3),
                         pooling='avg',
                         weights='imagenet')
for layer in pretrained_model.layers:
    layer.trainable = False
vgg_model.add(pretrained_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dense(9, activation='softmax'))
vgg_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 512)               262656    
                                                                 
 dense_5 (Dense)             (None, 9)                 4617      
                                                                 
Total params: 14981961 (57.15 MB)
Trainable params: 267273 (1.02 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
vgg_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
history = vgg_model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/10
10/10 [==============================] - 152s 15s/step - loss: 10.7126 - accuracy: 0.1368 - val_loss: 8.2628 - val_accuracy: 0.1711
Epoch 2/10
10/10 [==============================] - 150s 15s/step - loss: 3.7365 - accuracy: 0.4625 - val_loss: 4.9531 - val_accuracy: 0.3026
Epoch 3/10
10/10 [==============================] - 161s 16s/step - loss: 1.1298 - accuracy: 0.7459 - val_loss: 5.2775 - val_accuracy: 0.2763
Epoch 4/10
10/10 [==============================] - 161s 16s/step - loss: 0.4213 - accuracy: 0.8730 - val_loss: 3.8063 - val_accuracy: 0.3421
Epoch 5/10
10/10 [==============================] - 161s 16s/step - loss: 0.2034 - accuracy: 0.9251 - val_loss: 4.1399 - val_accuracy: 0.3026
Epoch 6/10
10/10 [==============================] - 161s 16s/step - loss: 0.0758 - accuracy: 0.9772 - val_loss: 4.1956 - val_accuracy: 0.2895
Epoch 7/10
10/10 [==============================] - 161s 16s/step - loss: 0.0330 - accuracy: 0.9935 - val_loss: 3.7237 - val_accuracy: 0.4079
Epoch

#Data Augmentation

In [ ]:
!pip install Augmentor

In [ ]:
import os
from PIL import Image
import Augmentor

def augment_images(input_dir, output_dir, num_samples=200):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    pipeline = Augmentor.Pipeline(input_dir, output_directory=output_dir)
    pipeline.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
    pipeline.flip_left_right(probability=0.5)
    pipeline.flip_top_bottom(probability=0.5)
    pipeline.zoom_random(probability=0.5, percentage_area=0.8)
    pipeline.flip_random(probability=0.5)
    pipeline.sample(num_samples)

def augment_all_brands(main_dir, output_base_dir, num_samples=200):
    for brand_dir in os.listdir(main_dir):
        brand_path = os.path.join(main_dir, brand_dir)
        if os.path.isdir(brand_path):
            output_dir = os.path.join(output_base_dir, f"augmented_{brand_dir}")
            print(f"Augmenting images in {brand_path}...")
            augment_images(brand_path, output_dir, num_samples)
def main():
    main_dir = '/content/drive/MyDrive/logos'
    output_base_dir = '/content/drive/MyDrive/augmented_logo'
    augment_all_brands(main_dir, output_base_dir, num_samples=200)
if __name__ == "__main__":
    main()

Augmenting images in /content/drive/MyDrive/logos/Volkswagen...
Initialised with 42 image(s) found.
Output directory set to /content/drive/MyDrive/augmented_logo/augmented_Volkswagen.

Processing <PIL.Image.Image image mode=RGB size=1000x750 at 0x79B81D9011B0>: 100%|██████████| 200/200 [00:24<00:00,  8.02 Samples/s]


Augmenting images in /content/drive/MyDrive/logos/Suzuki...
Initialised with 41 image(s) found.
Output directory set to /content/drive/MyDrive/augmented_logo/augmented_Suzuki.

Processing <PIL.Image.Image image mode=RGB size=331x248 at 0x79B7934FE200>: 100%|██████████| 200/200 [00:26<00:00,  7.69 Samples/s]


Augmenting images in /content/drive/MyDrive/logos/Nissan...
Initialised with 43 image(s) found.
Output directory set to /content/drive/MyDrive/augmented_logo/augmented_Nissan.

...

Processing <PIL.Image.Image image mode=RGB size=1473x982 at 0x79B7934FC940>: 100%|██████████| 200/200 [00:40<00:00,  4.93 Samples/s]


Augmenting images in /content/drive/MyDrive/logos/Ford...
Initialised with 43 image(s) found.
Output directory set to /content/drive/MyDrive/augmented_logo/augmented_Ford.

Processing <PIL.Image.Image image mode=RGB size=1062x795 at 0x79B792B556F0>: 100%|██████████| 200/200 [00:33<00:00,  6.00 Samples/s]


Augmenting images in /content/drive/MyDrive/logos/Toyota...
Initialised with 44 image(s) found.
Output directory set to /content/drive/MyDrive/augmented_logo/augmented_Toyota.

Processing <PIL.Image.Image image mode=RGB size=528x400 at 0x79B794625210>: 100%|██████████| 200/200 [00:20<00:00,  9.84 Samples/s]


In [ ]:
import os
for dirpath, dirnames, filenames in os.walk("/content/drive/MyDrive/augmented_logo"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

There are 9 directories and 0 images in '/content/drive/MyDrive/augmented_logo'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Volkswagen'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Suzuki'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Nissan'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Honday'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Hyundai'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Tata'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Renault'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Ford'.
There are 0 directories and 200 images in '/content/drive/MyDrive/augmented_logo/augmented_Toyota'.


In [ ]:
data_dir='/content/drive/MyDrive/augmented_logo'

In [ ]:
img_height,img_width=180,180
batch_size=32
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1800 files belonging to 9 classes.
Using 1440 files for training.


In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1800 files belonging to 9 classes.
Using 360 files for validation.


In [ ]:
class_names = val_ds.class_names
print(class_names)

['augmented_Ford', 'augmented_Honday', 'augmented_Hyundai', 'augmented_Nissan', 'augmented_Renault', 'augmented_Suzuki', 'augmented_Tata', 'augmented_Toyota', 'augmented_Volkswagen']


#RESNET50 MODEL

In [ ]:
resnet_model = Sequential()

pretrained_model= tf.keras.applications.ResNet50(include_top=False,
                   input_shape=(180,180,3),
                   pooling='avg',classes=9,
                   weights='imagenet')
for layer in pretrained_model.layers:
        layer.trainable=False

resnet_model.add(pretrained_model)
resnet_model.add(Flatten())
resnet_model.add(Dense(512, activation='relu'))
resnet_model.add(Dense(9, activation='softmax'))

In [ ]:
resnet_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 module_wrapper_3 (ModuleWr  (None, 2048)              0         
 apper)                                                          
                                                                 
 module_wrapper_4 (ModuleWr  (None, 512)               1049088   
 apper)                                                          
                                                                 
 module_wrapper_5 (ModuleWr  (None, 9)                 4617      
 apper)                                                          
                                                                 
Total params: 24641417 (94.00 MB)
Trainable params: 1053705 (4.02 MB)
Non-trainable params: 23587712 (89.98 MB)
________

In [ ]:
resnet_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
history = resnet_model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/10
45/45 [==============================] - 267s 6s/step - loss: 2.1388 - accuracy: 0.3597 - val_loss: 1.2552 - val_accuracy: 0.6056
Epoch 2/10
45/45 [==============================] - 290s 6s/step - loss: 0.8844 - accuracy: 0.7278 - val_loss: 0.9684 - val_accuracy: 0.7028
Epoch 3/10
45/45 [==============================] - 283s 6s/step - loss: 0.4766 - accuracy: 0.8590 - val_loss: 0.6202 - val_accuracy: 0.8250
Epoch 4/10
45/45 [==============================] - 282s 6s/step - loss: 0.2467 - accuracy: 0.9514 - val_loss: 0.4783 - val_accuracy: 0.8639
Epoch 5/10
45/45 [==============================] - 283s 6s/step - loss: 0.1279 - accuracy: 0.9833 - val_loss: 0.4563 - val_accuracy: 0.8500
Epoch 6/10
45/45 [==============================] - 281s 6s/step - loss: 0.0636 - accuracy: 0.9993 - val_loss: 0.3924 - val_accuracy: 0.8889
Epoch 7/10
45/45 [==============================] - 251s 6s/step - loss: 0.0375 - accuracy: 1.0000 - val_loss: 0.3618 - val_accuracy: 0.8972
Epoch 8/10
45

#Testing

In [ ]:
import cv2
image=cv2.imread("/content/drive/MyDrive/augmented_logo/augmented_Tata/Tata_original_frontfacingTatamotorscar0.jpeg_02338843-5c0b-48a6-bd89-087bab2d957f.jpeg")
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)


(1, 180, 180, 3)


In [ ]:
pred=resnet_model.predict(image)
print(pred)

1/1 [==============================] - 1s 1s/step
[[2.4799647e-02 1.1779516e-03 6.4664632e-03 5.8153863e-03 1.8649459e-02
  1.8289686e-03 9.4052708e-01 4.3679669e-04 2.9822518e-04]]


In [ ]:
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)

The predicted class is augmented_Tata


#VGG16 MODEL

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.applications import VGG16
import tensorflow as tf

In [ ]:
vgg_model = Sequential()
pretrained_model = VGG16(include_top=False,
                         input_shape=(180, 180, 3),
                         pooling='avg',
                         weights='imagenet')
for layer in pretrained_model.layers:
    layer.trainable = False
vgg_model.add(pretrained_model)
vgg_model.add(Flatten())
vgg_model.add(Dense(512, activation='relu'))
vgg_model.add(Dense(9, activation='softmax'))
vgg_model.summary()

58889256/58889256 [==============================] - 0s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 512)               262656    
                                                                 
 dense_1 (Dense)             (None, 9)                 4617      
                                                                 
Total params: 14981961 (57.15 MB)
Trainable params: 267273 (1.02 MB)
Non-trainable params: 14714688 (56.13 MB)
_________________________________________________________________


In [ ]:
vgg_model.compile(optimizer=Adam(lr=0.001),loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
epochs=10
history = vgg_model.fit(train_ds,validation_data=val_ds,epochs=epochs)

Epoch 1/10
45/45 [==============================] - 786s 17s/step - loss: 4.8777 - accuracy: 0.3757 - val_loss: 2.1831 - val_accuracy: 0.5667
Epoch 2/10
45/45 [==============================] - 714s 16s/step - loss: 0.9385 - accuracy: 0.7542 - val_loss: 1.0700 - val_accuracy: 0.7250
Epoch 3/10
45/45 [==============================] - 754s 17s/step - loss: 0.2409 - accuracy: 0.9208 - val_loss: 0.8816 - val_accuracy: 0.7500
Epoch 4/10
45/45 [==============================] - 695s 15s/step - loss: 0.0791 - accuracy: 0.9736 - val_loss: 0.6808 - val_accuracy: 0.8222
Epoch 5/10
45/45 [==============================] - 694s 15s/step - loss: 0.0200 - accuracy: 0.9986 - val_loss: 0.5809 - val_accuracy: 0.8306
Epoch 6/10
45/45 [==============================] - 698s 16s/step - loss: 0.0071 - accuracy: 1.0000 - val_loss: 0.5757 - val_accuracy: 0.8500
Epoch 7/10
45/45 [==============================] - 698s 16s/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.5687 - val_accuracy: 0.8472
Epoch 

In [ ]:
import cv2
image=cv2.imread("/content/drive/MyDrive/augmented_logo/augmented_Renault/Renault_original_frontfacingRenaultcar347.jpeg_21536322-eb53-48f7-8b1b-f0b89dbf209a.jpeg")
image_resized= cv2.resize(image, (img_height,img_width))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)

(1, 180, 180, 3)


In [ ]:
pred=vgg_model.predict(image)
print(pred)

1/1 [==============================] - 1s 692ms/step
[[2.1439398e-12 5.3616518e-14 3.1266800e-09 2.0911183e-12 9.9999988e-01
  1.0601261e-07 1.3246598e-09 1.3163775e-14 3.7392276e-13]]


In [ ]:
output_class=class_names[np.argmax(pred)]
print("The predicted class is", output_class)

The predicted class is augmented_Renault
